# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# Load exported csv into data frame
vacation_df = pd.read_csv('../WeatherPy/WeatherData.csv')
vacation_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,hasaki,40,JP,1.584241e+09,45,35.73,140.83,52.0,5.82
1,qingyuan,63,CN,1.584241e+09,45,23.70,113.03,70.0,2.24
2,whyalla,72,AU,1.584241e+09,45,-33.03,137.58,66.2,10.29
3,ushuaia,40,AR,1.584241e+09,62,-54.80,-68.30,53.6,8.05
4,hay river,75,CA,1.584241e+09,40,60.82,-115.80,15.8,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Store lat and lng in locations
locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"].astype(float)
max_humidity = Humidity.max()

# Configure gmaps
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity=max_humidity,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Narrow down the cities to fit specific weather conditions
vacation_conditions = vacation_df.loc[(vacation_df["Cloudiness"] == 0) & 
                                (vacation_df["Max Temp"] < 80) & 
                                (vacation_df["Max Temp"] > 70) &
                                (vacation_df["Windspeed"] < 10) , :]

vacation_conditions.reset_index(inplace=True)
del vacation_conditions['index']

# Print narrowed results
print(f" There are {len(vacation_conditions)} cities that fit the specified weather criteria.")
vacation_conditions

 There are 7 cities that fit the specified weather criteria.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,morondava,0,MG,1.584241e+09,88,-20.28,44.28,70.38,3.69
1,conakry,0,GN,1.584241e+09,80,9.54,-13.68,76.98,3.51
2,oussouye,0,SN,1.584241e+09,57,12.48,-16.55,78.80,4.70
3,ranchuelo,0,CU,1.584241e+09,73,22.38,-80.15,77.00,9.17
4,qandala,0,SO,1.584241e+09,75,11.47,49.87,74.73,5.21
5,salur,0,IN,1.584241e+09,48,18.53,83.22,75.94,3.94
6,nauta,0,PE,1.584241e+09,94,-4.51,-73.58,75.56,1.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
# # Store into hotel_df
# hotel_df = pd.DataFrame(vacation_conditions, columns=["City", "Country", "Lat", "Lng"])

# # Add "Hotel Name" Column
# hotel_df["Hotel Name"] = ""
# hotel_df

hotels = []

for i in range(len(vacation_conditions)):
    lat = vacation_conditions.loc[i]['Lat']
    lng = vacation_conditions.loc[i]['Lng']
    
    # Set up parameters for search
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params = params)
    response = requested.json()
    
    try:
        print(f"Processing Record {i}, City Name: {vacation_df.loc[i]['City']}.")
        hotels.append(response['results'][0]['name'])
        
    except:
        #hotels.append(np.nan)
        print("City not found...Skipping...")
        
print("-----------------------------")        
print("Data Retrieval Complete")
print("-----------------------------")

Processing Record 0, City Name: hasaki.
Processing Record 1, City Name: qingyuan.
Processing Record 2, City Name: whyalla.
Processing Record 3, City Name: ushuaia.
Processing Record 4, City Name: hay river.
Processing Record 5, City Name: vaini.
Processing Record 6, City Name: yellowknife.
-----------------------------
Data Retrieval Complete
-----------------------------


In [89]:
# Store data in hotel_df

hotel_df = vacation_conditions.copy()
hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df.dropna()
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed,Hotel Name
0,morondava,0,MG,1.584241e+09,88,-20.28,44.28,70.38,3.69,Morondava
1,conakry,0,GN,1.584241e+09,80,9.54,-13.68,76.98,3.51,Conakry
2,oussouye,0,SN,1.584241e+09,57,12.48,-16.55,78.80,4.70,Oussouye
3,ranchuelo,0,CU,1.584241e+09,73,22.38,-80.15,77.00,9.17,Ranchuelo
4,qandala,0,SO,1.584241e+09,75,11.47,49.87,74.73,5.21,Qandala
5,salur,0,IN,1.584241e+09,48,18.53,83.22,75.94,3.94,Salur
6,nauta,0,PE,1.584241e+09,94,-4.51,-73.58,75.56,1.14,A.h Marginal Manuel Pacaya


In [92]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [93]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))